In [1]:
import os
import argparse
import json
import sys
import torch
import numpy as np
import random

In [2]:
sys.path.insert(0, "Snippext_public")

#os.environ["CUDA_VISIBLE_DEVICES"] = ""

from ditto_light.dataset import DittoDataset
from ditto_light.summarize import Summarizer
from ditto_light.knowledge import *
from ditto_light.ditto import train

/home/kbryan/miniconda3/envs/ditto_3_8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import argparse
import sys

# Simulate command-line arguments in a notebook
sys.argv = [
    "notebook",
    "--task", "Structured/ditto_aircraft",
    "--batch_size", "64",
    "--max_len", "64",
    "--lr", "3e-5",
    "--n_epochs", "20",
    "--lm", "distilbert",
    "--fp16"
]

In [4]:
    parser = argparse.ArgumentParser()
    parser.add_argument("--task", type=str, default="Structured/Beer")
    parser.add_argument("--run_id", type=int, default=0)
    parser.add_argument("--batch_size", type=int, default=64)
    parser.add_argument("--max_len", type=int, default=64)
    parser.add_argument("--lr", type=float, default=3e-5)
    parser.add_argument("--n_epochs", type=int, default=20)
    parser.add_argument("--finetuning", dest="finetuning", action="store_true")
    parser.add_argument("--save_model", dest="save_model", action="store_true")
    parser.add_argument("--logdir", type=str, default="checkpoints/")
    parser.add_argument("--lm", type=str, default='distilbert')
    parser.add_argument("--fp16", dest="fp16", action="store_true")
    parser.add_argument("--da", type=str, default=None)
    parser.add_argument("--alpha_aug", type=float, default=0.8)
    parser.add_argument("--dk", type=str, default=None)
    parser.add_argument("--summarize", dest="summarize", action="store_true")
    parser.add_argument("--size", type=int, default=None)

    hp = parser.parse_args()

In [5]:
hp

Namespace(alpha_aug=0.8, batch_size=64, da=None, dk=None, finetuning=False, fp16=True, lm='distilbert', logdir='checkpoints/', lr=3e-05, max_len=64, n_epochs=20, run_id=0, save_model=False, size=None, summarize=False, task='Structured/ditto_aircraft')

In [6]:
    # set seeds
    seed = hp.run_id
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    # only a single task for baseline
    task = hp.task

    # create the tag of the run
    run_tag = '%s_lm=%s_da=%s_dk=%s_su=%s_size=%s_id=%d' % (task, hp.lm, hp.da,
            hp.dk, hp.summarize, str(hp.size), hp.run_id)
    run_tag = run_tag.replace('/', '_')

    # load task configuration
    configs = json.load(open('configs.json'))
    configs = {conf['name'] : conf for conf in configs}
    config = configs[task]

    trainset = config['trainset']
    validset = config['validset']
    testset = config['testset']

In [7]:
hp.run_id

0

In [8]:
    # load train/dev/test sets
    train_dataset = DittoDataset(trainset,
                                   lm=hp.lm,
                                   max_len=hp.max_len,
                                   size=hp.size,
                                   da=hp.da)
    valid_dataset = DittoDataset(validset, lm=hp.lm)
    test_dataset = DittoDataset(testset, lm=hp.lm)


/home/kbryan/miniconda3/envs/ditto_3_8/lib/python3.8/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


RuntimeError: Failed to import transformers.models.distilbert.configuration_distilbert because of the following error (look up to see its traceback):
Failed to import transformers.onnx.config because of the following error (look up to see its traceback):
libtiff.so.5: cannot open shared object file: No such file or directory

In [ ]:
train_dataset.pairs

NameError: name 'train_dataset' is not defined

In [ ]:
# train and evaluate the model
train(train_dataset,
        valid_dataset,
        test_dataset,
        run_tag, hp)